# Lecture: Tests

**Week 11**

- based on Martin's lecture
  
[Testing](#Testing)

## Testing

* many ways to test code
* you've all done an exploratory/manual testing
* to cover the whole codebase with manual tests, it is necessary:
    * list all the code/projects features
    * collect all (different) types of inputs it 
    * collect the corresponding expected results
* !problem: change in code → change the above
    * not fun → **automated testing**
        * running test from script instead of manually
   
* 2 main test categories:
    * integration tests - testing multiple if multiple components work together
    * unit tests - testing a single component

* (most) functional tests consist of:
    1. **Arrange** - conditions in/for which we test
    2. **Act** - running the behaviour we want to test
    3. **Assert** - check if behaviour produced expected result
    4. **Cleanup** - don't influence other tests

* the most basic test can be done using `assert` method
    * e.g. lets check/test if `len` method is the same as `__len__`

In [ ]:
a_list = [1,2,3,5] 
assert len(a_list) == a_list.__len__(), "Function len returned different result than method __len__"

* we could try different data-structure

In [ ]:
a_tuple = (1,2,3,5)
assert len(a_tuple) == a_tuple.__len__(), "Function len returned different result than method __len__"

In [ ]:
a_tuple = (1,2,3,5)
assert len(a_tuple) == a_tuple.__len__(), "Function len returned different result than method __len__"

In [ ]:
assert sum([1,1]) == 3, '2. Your result is off.'

* instead of testing on the REPL, we can put our tests into a test script and run it 

In [ ]:
# %load test_1.py
def test_sum():
    assert sum([1,1]) == 2, "Should be 2"
    
def test_len_vs__len__():
    a_tuple = (1,2,3,5)
    assert len(a_tuple) == a_tuple.__len__(), "Function len returned differnt result than method __len__"
    
if __name__ == "__main__":
    test_sum()
    test_len_vs__len__()
    print('All tests passed.')

In [ ]:
%run test_1.py

* OK for simple check, cumbersome for more tests
    * → **test runners**
* test runner = application designed for running tests
    * check the output
    * offer tools for diagnosing
    
* many test runners available for Python
    * *unittest* (built into the Python standard library)
    * nose/nose2
    * doctest
    * robot
    * **pytest**, ...


## pytest

* a framework for building simple and scalable tests
* one of the most popular Python testing frameworks
    * feature-rich
    * a lot of available [plugins](https://docs.pytest.org/en/latest/reference/plugin_list.html)
 
* pytest works with the simple assert statements
    * not necessarily the case with other test runners

* how does pytest know which tests to run?
    * by default it runs all files of the form `test_*.py` or `*_test.py` in the current directory and subdirectories
        * however check [conventions for test discovery rules](https://docs.pytest.org/en/6.2.x/goodpractices.html#test-discovery)

In [ ]:
!pytest

* what does it tell us:
    * the system tests are run on (Python, pytest version, and any pluggins
    * *rootdir* : where are we running things from
    * [XX%] next to each test script shows success rate of all tests
    * it will show you a failure report with detailed explanation (not here)
        * lets fail

In [ ]:
#%%writefile test_2.py
#%%read test_2.py

def test_sum():
    assert sum([1,1]) == 3, "Should be 2"

def test_len_vs__len__():
    a_tuple = (1,2,3,5)
    assert len(a_tuple) == a_tuple.__len__(), "Function len returned differnt result than method __len__"

In [ ]:
!pytest

* output next to the script indecates the status of each test:
    * "." - test passed
    * "F" - test failed
    * "E" - test raised an unexcpected exception

* it does not only show you the AssertionError though
    * what does it show us (compared to the simple assert statement)?

* if we want to run only some tests, we can specify which to ignore
    * `--ignore`
    * `--ignore-glob` - using glob (wildcard like patterns)

In [ ]:
%%writefile tests/test_3.py

def test_sum():
    assert sum([1,1]) == 3, "Should be 2"

def test_len_vs__len__():
    a_tuple = (1,2,3,5)
    assert len(a_tuple) == a_tuple.__len__(), "Function len returned differnt result than method __len__"

In [ ]:
# checking where we are
#!cd
!pwd

In [ ]:
!pytest --ignore=tests/

In [ ]:
# when not ignoring
!pytest --ignore-glob='*_3.py'

* in most modern code editors, managing a set of tests is more user friendly than from command line


* tests often depend on:
    * data
    * test doubles
* we don't want to mess with the originals => pytest **fixtures**

### Fixtures
* "arranging" part of the test

* a method for providing:
    * data
    * test doubles
    * state setup 

* more tests using the same underlying dataset → use fixture
     * (repeating) data provided by a single function [decorated](#Decorators) with `@pytest.fixture`
     
* test depending on a fixture needs to have a fixture as an argument

* let's look at the test double first

In [ ]:
# !cd
!pwd

In [ ]:
# %load tests/test_fixture_smtp.py
import pytest

@pytest.fixture
def smtp():
    """Initialize and return SMTP client session object"""
    import smtplib
    return smtplib.SMTP("smtp.gmail.com")

def test_ehlo(smtp):
    """Test response from sending Extended Helo (EHLO) is 250."""
    response, msg = smtp.ehlo()
    assert response == 250
    # assert 0 

In [ ]:
!pytest tests/test_fixture_smtp.py

* now fixture for providing data
    * note: when providing path, think about the sourcedirectory! 

In [ ]:
# %load tests/test_fixtures_data.py
import pytest 

@pytest.fixture
def data_names():
    import pandas as pd
    df = pd.read_csv('data/test_data_names.csv')
    return df

def test_addressing(data_names):
    df = data_names
    titles = df['Title']
    surnames = df['Surname']
    expected = df[['Addressing']]
    assert (titles + ' ' + expected == surnames).all()

In [ ]:
!pytest tests/test_fixtures_data.py

* when to avoid fixtures:
    * using fixtures fixtures is as bad as using tests redundantly
    *  => **marks**

### Marks - test filtering

* you might want to only run couple of your tests
    * full suite of tests only sometimes
    
* to filter which tests to run:
    * name-based filtering
    * directory scoping 
    * **test categorization** (`-m` parameter)
    
* create **marks** (custom labels) to label any test you like (can have multiple labels)
    * e.g. you can categorize your tests by dependencies (e.g. access to database - could be `@pytest.mark.database_access`
* to run only tests in specific category (mark) `pytest -m <mark>`
* to *not* run tests with specific mark `pytest -m "not <mark>"`

* you should also [register the custom markers](https://stackoverflow.com/questions/60806473/pytestunknownmarkwarning-unknown-pytest-mark-xxx-is-this-a-typo) in *pytest.ini* file

In [ ]:
# %load tests/test_mark_example.py
import pytest 

@pytest.mark.database
def test_pg_read():
    pass

@pytest.mark.database
def test_pg_write():
    pass

In [ ]:
!pytest -m database

* there are few marks out of the box:
    * **skip** skips a test unconditionally
    * **skipif** skips a test if the expression passed to it evaluates to True
    * **parametrize** creates multiple variants of a test with different values as arguments
    
* you can see a list of all the marks pytest knows about by running `pytest --markers`

In [ ]:
!pytest --markers

### Test parametrization

* using only slightly different input and output would lead to repeating test definitions
    * DRY!
* fixtures not very good with only slightly different inputs and expected outputs
    * **parametrize** a single test definition a get variants of the test for you with the parameters you specify
    * mind the syntax


In [ ]:
# %load tests/test_parametrize_example.py
import pytest
import unicodedata

#######
# Function we would like to test should be defined in package code, not here.
########
def drop_diacritics(text: str) -> str:
    """
    Strip accents from input String.
    
    :param text: The input string.
    :returns: The processed string.
    """
    if not isinstance(text, str):
        raise TypeError(f'Input text should be a string, not %s', type(text))
    
    # Return the normal form for the Unicode string
    # 'NFKD' stands for the normal form KD  
    text = unicodedata.normalize('NFKD',text)
    output = ''
    
    for char in text:
        if not unicodedata.combining(char):
            output += char
            
    return output
#### 


@pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
def test_eval(test_input, expected):
    assert eval(test_input) == expected
    
@pytest.mark.parametrize(
    'original,output',
    [
        ('řeřicha', 'rericha'),
        ('čeština', 'cestina')
    ]
) 
def test_drop_diacritics(original:str, output:str) -> None:
    assert drop_diacritics(original) == output
    

In [ ]:
!pytest tests/test_parametrize_example.py

### Testing features to explore

* [plugins](https://docs.pytest.org/en/latest/reference/plugin_list.html)
    * requests-mock
    * database-mock

* [CI/CD](https://docs.github.com/en/actions/guides/about-continuous-integration)
